In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from math import sqrt
from numpy.random import normal
import plotly.graph_objs as go
import plotly.express as px
import csv

In [ ]:
def indicesOfRatings(matrix):
    non_nan_indices = np.where(~np.isnan(matrix))
    return list(zip(non_nan_indices[0],non_nan_indices[1]))
    
def rmse(original,reproduction,indexSet):
    s=0
    for row,col in indexSet:
        s+=pow(original[row,col]-reproduction[row,col],2)
    s=s/len(indexSet)
    return sqrt(s)
    
def initializePQ(size,k):
    n,m=size
    P=normal(scale=.5,size=(n,k))
    Q=normal(scale=.5,size=(k,m))
    return P,Q

def epoch(original,p,q,stepsize,train,test):
    for row,col in train:
        err=original[row,col]-p[row,:]@q[:,col]
        p[row,:]=p[row,:]+stepsize*q[:,col].T*err
        err=original[row,col]-p[row,:]@q[:,col]
        q[:,col]=q[:,col]+stepsize*p[row,:].T*err
    trainerror=rmse(original,p@q,train)
    testerror=rmse(original,p@q,test)
    return p,q,trainerror,testerror

In [ ]:
dataset='ml-latest-small'

def getmovies():
    movies=dict() #make a dictionary that goes from movieID-1 to the movie name
    with open(dataset+'/movies.csv') as f:
        f.readline()
        reader=csv.reader(f,delimiter=',')
        for row in reader:
            key=int(row[0])
            movies[key]=row[1]
    return movies

def getratings():
    df=pd.read_csv(dataset+'/ratings.csv')
    df2=df.pivot(columns=['movieId'],index=['userId'],values=['rating'])
    return df2

In [ ]:
ratings=getratings()
movies=getmovies()
ratings